# Waveform Fitting of Receiver Functions
# (Using Multiple Processors)

This notebook demonstrates waveform fitting of receiver function to retrieve Moho structure based on [rfsed](https://github.com/akinremisa/rfsed) package

The required functions are first imported 

In [2]:
#%% Example of Waveform Fitting with Multiprocessor
import os
import numpy as np
from rfsed.rf import read_rf,rfstats,RFStream
from rfsed.WaveformFittingMultiproc import WaveformPara, run_waveformfitting, plotbestmodel
from rfsed.util import rfMoho_example, rfSed_example, save_plot
from obspy import read
from obspy.geodetics.base import kilometer2degrees

The receiver function data is defined and the Moho parameters. The waveform fitting analysis is performed and the results visualised

In [4]:
#----------------------------------------------------------
##load rf data into RFStream
stationname = "ZQ_174"
stream = read(f"../seismic_data_RFAC/{stationname}/*.BHR.sac_g5")
for tr in stream:
    slowness = tr.stats.sac['user0']/kilometer2degrees(1)
    # print(slowness)
    tr.stats.sac['user1']=slowness
    tr.stats.sac['user0'] = 0.0
    tr.stats.sac.setdefault('o',-1)
    tr.stats.sac.setdefault('a',0)
    tr.stats.sac.setdefault('kuser0','rf')
    tr.stats.sac.setdefault('kuser1','Ps')
    tr.stats.sac.setdefault('kuser2','Ps')
st_rf = RFStream(stream)

savedir = "../Hk_RFfit2layer"
savepath = savedir + f'/{stationname}'
if not os.path.exists(savepath):  # create data folder if necessary
    os.makedirs(savepath)

Yeck_result_file = f"../Hk_RF2layer/{stationname}/{stationname}_Hk_result.dat"
data_hk = np.loadtxt(Yeck_result_file,skiprows=1)
# rfstream= rfMoho_example()
preonset = 5
VpSed = data_hk[0,0]
KSed = data_hk[0,2]
Sedthick=data_hk[0,1]
VpCrust = data_hk[1,0]
gauparameter=5
KMoho= np.linspace(1.65,2.5,101)
HMoho=np.linspace(20,45,101)
# KMoho= np.linspace(1.65,1.95,5)
# HMoho=np.linspace(20,60,5)
rayp=0.06
delta = st_rf[0].stats.delta
wtCorr=0.5
wtRMSE=0.3
wtPG=0.2
format='png'
nproc=16
ModelParams=WaveformPara(st_rf, preonset, Sedthick, VpSed, KSed, VpCrust, rayp, KMoho, HMoho, gauparameter)
Results=run_waveformfitting(nproc, HMoho, ModelParams)
plotbestmodel(Results, ModelParams, wtCorr, wtRMSE, wtPG, savepath, format)

Best Model from Correlation Coefficient: Moho at 44.25 km with Vp/Vs of 1.69
Best Model from RMSE: Moho at 20.00 km with Vp/Vs of 1.76
Best Model from PG: Moho at 44.00 km with Vp/Vs of 2.49
Best Model from Overall Goodness of Fit: Moho at 44.75 km with Vp/Vs of 1.68
